In [1]:
# CSE545-SDG3-Matrix


In [2]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql.types import *
import numpy as np
import pandas as pd
import scipy as sc
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.sql.types import *
import os

In [3]:
sc = pyspark.SparkContext(appName="SDG3")
sqlCtx = SQLContext(sc)
sc.version

'2.2.0'

In [4]:
data_path = r'../LinkPE14US'
#print(os.path.join(data_path, "*.*PUB"))#all
print(os.path.join(data_path, "VS14LINK.USMERGEDPUB"))#*.*PUB
#df = sqlCtx.read.text(os.path.join(data_path, "*.*PUB"))  #all #*.*PUB
df = sqlCtx.read.text(os.path.join(data_path, "VS14LINK.USMERGEDPUB"))  #merged
#df = sqlCtx.read.text(os.path.join(data_path, "VS14LINK.USNUMPUB"))  #350 mb
#df = sqlCtx.read.text("/VS14LINK.USDENPUB") # 5gb
#df = sqlCtx.read.text("/*.*PUB") # both

df.count()


../LinkPE14US\VS14LINK.USMERGEDPUB


26085

In [5]:
#- eading

In [6]:
def blank_as_Y(x):
    return when(col(x) == '', 'Y').otherwise('N') # 1 == alive, 0 == demised for ICD10

In [7]:
# pandas data frame after adding column metadata
#pndf type = pyspark.sql.dataframe.DataFrame
pndf = df.select(
    df.value.substr(9,4).cast(IntegerType()).alias('Birth_Year'),
    df.value.substr(13,2).cast(IntegerType()).alias('Birth_Month'),
    df.value.substr(19,4).cast(IntegerType()).alias('Time_of_Birth'),
    df.value.substr(23,1).cast(IntegerType()).alias('Birth_DOW'),
    df.value.substr(32,1).cast(IntegerType()).alias('Birth_Place'),
    df.value.substr(50,1).cast(IntegerType()).alias('Facility_Recode'),
    df.value.substr(75,2).cast(IntegerType()).alias('Mothers_Age_Recode'),
    df.value.substr(84,1).cast(IntegerType()).alias('Mothers_Nativity'),
    df.value.substr(104,1).cast(IntegerType()).alias('Residence_Status'),
    df.value.substr(105,2).cast(IntegerType()).alias('Mothers_Race'),
    df.value.substr(110,1).cast(IntegerType()).alias('Bridge_Race_Mother'),
    df.value.substr(115,1).cast(IntegerType()).alias('Mothers_Hispanic_Origin_Recode'),
    df.value.substr(119,1).cast(IntegerType()).alias('Paternity_Ack'),
    df.value.substr(120,1).cast(IntegerType()).alias('Marital_Status'),
    df.value.substr(124,1).cast(IntegerType()).alias('Mothers_Education'),
    df.value.substr(147,2).cast(IntegerType()).alias('Fathers_Age'),
    df.value.substr(149,2).cast(IntegerType()).alias('Fathers_Age_Recode'),
    df.value.substr(151,2).cast(IntegerType()).alias('Fathers_Race_Recode'),
    df.value.substr(156,1).cast(IntegerType()).alias('Bridge_Race_Father'),
    df.value.substr(160,1).cast(IntegerType()).alias('Fathers_Hispanic_Origin_Recode'),
    df.value.substr(163,1).cast(IntegerType()).alias('Fathers_Education'),
    df.value.substr(171,2).cast(IntegerType()).alias('Prior_Births_Living'),
    df.value.substr(173,2).cast(IntegerType()).alias('Prior_Births_Dead'),
    df.value.substr(175,2).cast(IntegerType()).alias('Prior_Other_Terminations'),
    df.value.substr(179,1).cast(IntegerType()).alias('Live_Birth_Order_Recode'),
    df.value.substr(182,1).cast(IntegerType()).alias('Total_Birth_Order_Recode'),
    df.value.substr(198,3).cast(IntegerType()).alias('Interval_Last_Live_Birth_Recode'),
    df.value.substr(201,2).cast(IntegerType()).alias('Interval_Last_Live_Birth_Recode11'),
    df.value.substr(206,3).cast(IntegerType()).alias('Interval_Last_Other_Pregnancy_Recode'),
    df.value.substr(209,2).cast(IntegerType()).alias('Interval_Last_Other_Pregnancy_Recode11'),
    df.value.substr(214,3).cast(IntegerType()).alias('Interval_Last_Pregnancy_Recode'),
    df.value.substr(217,2).cast(IntegerType()).alias('Interval_Last_Pregnancy_Recode11'),
    df.value.substr(238,2).cast(IntegerType()).alias('Num_Prenatal_Visits'),
    df.value.substr(242,2).cast(IntegerType()).alias('Num_Prenatal_Visits_Recode'),
    df.value.substr(253,2).cast(IntegerType()).alias('Cigarettes_Before_Pregnancy'),
    df.value.substr(255,2).cast(IntegerType()).alias('Cigarettes_1st_Trimester'),
    df.value.substr(257,2).cast(IntegerType()).alias('Cigarettes_2nd_Trimester'),
    df.value.substr(259,2).cast(IntegerType()).alias('Cigarettes_3rd_Trimester'),
    df.value.substr(261,1).cast(IntegerType()).alias('Cigarettes_Before_Pregnancy_Recode'),
    df.value.substr(262,1).cast(IntegerType()).alias('Cigarettes_1st_Trimester_Recode'),
    df.value.substr(263,1).cast(IntegerType()).alias('Cigarettes_2nd_Trimester_Recode'),
    df.value.substr(264,1).cast(IntegerType()).alias('Cigarettes_3rd_Trimester_Recode'),
    df.value.substr(280,2).cast(IntegerType()).alias('Mothers_Height_Inches'),
    df.value.substr(287,1).cast(IntegerType()).alias('BMI_Recode'),
    df.value.substr(292,3).cast(IntegerType()).alias('Pre_Pregnancy_Weight_Recode'),
    df.value.substr(299,3).cast(IntegerType()).alias('Delivery_Weight_Recode'),
    df.value.substr(304,2).cast(IntegerType()).alias('Weight_Gain'),
    df.value.substr(306,1).cast(IntegerType()).alias('Weight_Gain_Recode'),
    df.value.substr(313,1).cast(StringType()).alias('Pre_Pregnancy_Diabetes'),
    df.value.substr(314,1).cast(StringType()).alias('Gestational_Diabetes'),
    df.value.substr(315,1).cast(StringType()).alias('Pre_Pregnancy_Hypertension'),
    df.value.substr(316,1).cast(StringType()).alias('Gestational_Hypertension'),
    df.value.substr(317,1).cast(StringType()).alias('Hypertension_Eclmapsia'),
    df.value.substr(318,1).cast(StringType()).alias('Previous_Preterm_Birth'),
    df.value.substr(325,1).cast(StringType()).alias('Infertility_Treatment'),
    df.value.substr(326,1).cast(StringType()).alias('Fertility_Enhancing_Drugs'),
    df.value.substr(327,1).cast(StringType()).alias('Assistive_Reproductive_Technology'),
    df.value.substr(331,1).cast(StringType()).alias('Prev_Cesareans'),
    df.value.substr(332,2).cast(IntegerType()).alias('Num_Prev_Cesareans'),
    df.value.substr(337,1).cast(StringType()).alias('No_Risk_Factors_Checked'),
    
    df.value.substr(343,1).cast(StringType()).alias('Gonorrhea'),
    df.value.substr(344,1).cast(StringType()).alias('Syphillis'),
    df.value.substr(345,1).cast(StringType()).alias('Chlamydia'),
    df.value.substr(346,1).cast(StringType()).alias('HepatitisB'),
    df.value.substr(347,1).cast(StringType()).alias('HepatitisC'),
    df.value.substr(353,1).cast(StringType()).alias('No_Infections_Checked'),
    df.value.substr(360,1).cast(StringType()).alias('Sucessful_External_Cephalic_Version'),
    df.value.substr(361,1).cast(StringType()).alias('Failed_External_Cephalic_Version'),
    
    df.value.substr(383,1).cast(StringType()).alias('Induction_Of_Labor'),
    df.value.substr(384,1).cast(StringType()).alias('Augmentation_Of_Labor'),
    df.value.substr(385,1).cast(StringType()).alias('Steroids_Mother'),
    df.value.substr(386,1).cast(StringType()).alias('Antibiotics_Mother'),
    df.value.substr(386,1).cast(StringType()).alias('Chorioamnionitis_Mother'),
    df.value.substr(387,1).cast(StringType()).alias('Anesthesia_Mother'),
    df.value.substr(395,1).cast(StringType()).alias('No_Labor_Checked'),
    df.value.substr(401,1).cast(IntegerType()).alias('Fetal_Presentation'),
    df.value.substr(402,1).cast(IntegerType()).alias('Final_Route_and_Method_Of_Delivery'),
    df.value.substr(403,1).cast(IntegerType()).alias('Trail_Of_Labor_Attempted'),
    df.value.substr(407,1).cast(IntegerType()).alias('Delivery_Method_Recode'),
    df.value.substr(415,1).cast(StringType()).alias('Maternal_Transfusion'),
    df.value.substr(416,1).cast(StringType()).alias('Perineal_Laceration'),
    df.value.substr(417,1).cast(StringType()).alias('Ruptured_Uterus'),
    df.value.substr(418,1).cast(StringType()).alias('Unplanned_Hysterectomy'),
    df.value.substr(419,1).cast(StringType()).alias('Admit_to_Intensive_Care'),
    df.value.substr(427,1).cast(IntegerType()).alias('No_Maternal_Morbidity_Checked'),
    df.value.substr(433,1).cast(IntegerType()).alias('Attendant'),
    df.value.substr(434,1).cast(StringType()).alias('Mother_Transferred'),

    
    #435 - 870
    df.value.substr(435,1).cast(IntegerType()).alias('Payment_Source'),
    df.value.substr(436,1).cast(IntegerType()).alias('Payment_Recode'),
    df.value.substr(444,2).cast(IntegerType()).alias('Five_Minute_APGAR_Score'),
    df.value.substr(446,1).cast(IntegerType()).alias('Five_Minute_APGAR_Recode'),
    df.value.substr(448,2).cast(IntegerType()).alias('Ten_Minute_APGAR_Score'),
    df.value.substr(450,1).cast(IntegerType()).alias('Ten_Minute_APGAR_Recode'),
    df.value.substr(454,1).cast(IntegerType()).alias('Plurality'),
    df.value.substr(475,1).cast(StringType()).alias('Sex_Of_Infant'),
    df.value.substr(477,2).cast(IntegerType()).alias('Last_Normal_Menses_Month'),
    df.value.substr(481,4).cast(IntegerType()).alias('Last_Normal_Menses_Year'),
    df.value.substr(489,1).cast(IntegerType()).alias('Obstetric_Estimate_of_Gestation_Used_Flag'),
    df.value.substr(490,2).cast(IntegerType()).alias('Combined_Gestation_Detail_in_Weeks'),
    df.value.substr(492,1).cast(IntegerType()).alias('Combined_Gestation_Recode_Weeks'),
    df.value.substr(509,2).cast(IntegerType()).alias('Birth_Weight_Recode_14'),
    df.value.substr(511,1).cast(IntegerType()).alias('Birth_Weight_Recode_4'),
    df.value.substr(512,4).cast(IntegerType()).alias('Imputed_Birthwieght'),
    #Abnormal Conditions of the Newborn
    df.value.substr(517,1).cast(StringType()).alias('Assisted_Ventilation'),
    df.value.substr(518,1).cast(StringType()).alias('Assisted_Ventilation_Greater_Than_6_Hours'),
    df.value.substr(519,1).cast(StringType()).alias('Admission_to_NICU'),
    df.value.substr(520,1).cast(StringType()).alias('Surfactant'),
    df.value.substr(521,1).cast(StringType()).alias('Antibiotics'),
    df.value.substr(522,1).cast(StringType()).alias('Seizures'),
    df.value.substr(531,1).cast(IntegerType()).alias('No_Abnormal_Conditions_Checked'),
    #Congenital Anomalies of the Newborn
    df.value.substr(537,1).cast(StringType()).alias('Anencephaly'),
    df.value.substr(538,1).cast(StringType()).alias('Meningomyelocele_or_Spina_Bifida'),
    df.value.substr(539,1).cast(StringType()).alias('Cyanotic_Congenital_Heart_Disease'),
    df.value.substr(540,1).cast(StringType()).alias('Congenital_Diaphragmatic_Hernia'),
    df.value.substr(541,1).cast(StringType()).alias('Omphalocele'),
    df.value.substr(542,1).cast(StringType()).alias('Gastroschisis'),
    df.value.substr(549,1).cast(StringType()).alias('Limb_Reduction_Defect'),
    df.value.substr(550,1).cast(StringType()).alias('Cleft_Lip_w_or_wo_Cleft_Palate'),
    df.value.substr(551,1).cast(StringType()).alias('Cleft_Palate_alone'),
    df.value.substr(553,1).cast(StringType()).alias('Suspected_Chromosomal_Disorder'),
    df.value.substr(554,1).cast(StringType()).alias('Hypospadias'),
    df.value.substr(561,1).cast(IntegerType()).alias('No_Congenital_Anomalies_Checked'),
    df.value.substr(568,1).cast(StringType()).alias('Infant_Living_at_Time_of_Report'),
    df.value.substr(569,1).cast(StringType()).alias('Infant_Being_Breastfed'), 
    #
    #1330 - 1743
    df.value.substr(1331,1).cast(IntegerType()).alias('Diabetes'),
    df.value.substr(1332,1).cast(IntegerType()).alias('Chronic_Hypertension'),
    df.value.substr(1333,1).cast(IntegerType()).alias('Prepregnacy_Associated_Hypertension'),
    df.value.substr(1334,1).cast(IntegerType()).alias('Eclampsia'),
    df.value.substr(1335,1).cast(IntegerType()).alias('Forceps'),
    df.value.substr(1336,1).cast(IntegerType()).alias('Vacuum'),
    df.value.substr(1337,1).cast(IntegerType()).alias('Induction_of_Labor1'),
    df.value.substr(1338,1).cast(IntegerType()).alias('Breech1'),
    df.value.substr(1341,1).cast(IntegerType()).alias('Spina_Bifida_Meningocele1'),
    df.value.substr(1342,1).cast(IntegerType()).alias('Omphalocele_Gastroschisis1'),
    
    df.value.substr(1343,1).cast(IntegerType()).alias('Cleft_Lip_Palate1'), 
    df.value.substr(1344,1).cast(IntegerType()).alias('Diaphragmatic_Herni1a'),
    df.value.substr(1345,1).cast(IntegerType()).alias('Down_Syndrome1'),
    df.value.substr(1356,3).cast(IntegerType()).alias('Age_at_Death_in_Days'),
    df.value.substr(1359,1).cast(IntegerType()).alias('Infant_age_recode_5'),
    df.value.substr(1360,2).cast(IntegerType()).alias('Infant_age_recode_22'), 
    df.value.substr(1362,1).cast(IntegerType()).alias('Manner_of_Death'),
    
    df.value.substr(1366,1).cast(IntegerType()).alias('Place_of_injury_for_causes_W00_Y34_except_Y06'),
    df.value.substr(1368,4).cast(StringType()).alias('ICD10'), #underlying cause of death - this is used to determine the LABEL for ML
    df.value.substr(1671,1).cast(IntegerType()).alias('Day_of_Week_of_Death'),
    df.value.substr(1672,4).cast(IntegerType()).alias('Data_Year'),
    df.value.substr(1742,1).cast(IntegerType()).alias('Month_of_Death')
    
    
).withColumn("id", monotonically_increasing_id())

pndf.printSchema
pndf = pndf.na.fill({'Num_Prev_Cesareans': 0.0})

# derive the infant_living column from ICD10 - must for ML prediction
ndf = pndf.withColumn("Infant_Living", blank_as_Y("ICD10")) # dervied 0/1 label  # 1 == alive, 0 == demised for ICD10
print("number of features ", len(df.columns))

number of features  1


In [8]:
# how many infants demised
ndf.filter(ndf['Infant_Living']=='N').count()

23085

In [9]:
ndf.filter(ndf['Birth_Year'] == 2014).count()

23412

In [10]:
def blank_as_null(x):
    return when(col(x) != ' ', col(x)).otherwise(np.nan)


In [11]:
# pandas data frame after adding column metadata

ndf.printSchema
pndf = ndf.na.fill("NA")
pndf = pndf.toPandas()
#pndf.replace("", np.nan, regex=True)
pndf.head(30)

,Birth_Year,Birth_Month,Time_of_Birth,Birth_DOW,Birth_Place,Facility_Recode,Mothers_Age_Recode,Mothers_Nativity,Residence_Status,Mothers_Race,...,Infant_age_recode_5,Infant_age_recode_22,Manner_of_Death,Place_of_injury_for_causes_W00_Y34_except_Y06,ICD10,Day_of_Week_of_Death,Data_Year,Month_of_Death,id,Infant_Living
0,2013.0,7.0,1853.0,3.0,1.0,1.0,24.0,1,1.0,NaN,...,5.0,16.0,5.0,NaN,R99,7.0,2014.0,0.0,0,N
1,2013.0,9.0,841.0,3.0,1.0,1.0,27.0,1,1.0,NaN,...,5.0,16.0,5.0,NaN,R95,7.0,2014.0,0.0,1,N
2,2013.0,10.0,829.0,3.0,1.0,1.0,25.0,1,2.0,NaN,...,5.0,13.0,5.0,NaN,R99,4.0,2014.0,0.0,2,N
3,2013.0,10.0,1913.0,5.0,1.0,1.0,24.0,1,1.0,NaN,...,5.0,14.0,5.0,NaN,R99,4.0,2014.0,0.0,3,N
4,2013.0,10.0,2055.0,4.0,1.0,1.0,32.0,1,1.0,NaN,...,5.0,19.0,5.0,NaN,R99,5.0,2014.0,0.0,4,N
5,2013.0,11.0,2156.0,5.0,1.0,1.0,29.0,1,1.0,NaN,...,5.0,14.0,7.0,NaN,G129,1.0,2014.0,0.0,5,N
6,2013.0,11.0,11.0,2.0,1.0,1.0,19.0,1,1.0,NaN,...,5.0,14.0,7.0,NaN,A419,3.0,2014.0,0.0,6,N
7,2013.0,12.0,815.0,2.0,1.0,1.0,28.0,1,1.0,NaN,...,5.0,18.0,1.0,0.0,W75,6.0,2014.0,0.0,7,N
8,2013.0,12.0,1114.0,4.0,1.0,1.0,22.0,1,1.0,NaN,...,5.0,12.0,5.0,NaN,R95,4.0,2014.0,0.0,8,N
9,2013.0,12.0,2330.0,4.0,1.0,1.0,22.0,1,1.0,NaN,...,4.0,10.0,7.0,NaN,P77,3.0,2014.0,0.0,9,N


In [12]:
# how many infants demised
pndf.filter(pndf['Infant_Living']=='N').count()

Series([], dtype: int64)

In [13]:
pndf.shape

(26085, 148)

In [14]:
#featureDf = ndf.withColumn("Delivery_Weight_gms", toInt(df['Delivery_Weight_lbs'])).select('Delivery_Weight_lbs')

In [15]:
#pndf[pd.isnull(pndf['Infant_Living'])]
#pndf[pndf.Infant_Living == None]
pndf.Infant_Living.unique()

array(['N', 'Y'], dtype=object)

In [16]:
#pndf.ICD10.unique()

In [17]:
pndf.hist(figsize=(148, 136), alpha=0.5, bins=50)
plt.show()

In [18]:
pndf.describe()

,Birth_Year,Birth_Month,Time_of_Birth,Birth_DOW,Birth_Place,Facility_Recode,Mothers_Age_Recode,Mothers_Nativity,Residence_Status,Mothers_Race,...,Down_Syndrome1,Age_at_Death_in_Days,Infant_age_recode_5,Infant_age_recode_22,Manner_of_Death,Place_of_injury_for_causes_W00_Y34_except_Y06,Day_of_Week_of_Death,Data_Year,Month_of_Death,id
count,26084.000000,26084.000000,25238.000000,26084.000000,25237.000000,26084.000000,26084.000000,26085.000000,26084.000000,22904.000000,...,26085.000000,23085.000000,23085.000000,23085.000000,20123.000000,1417.000000,23085.000000,23085.0,23085.000000,2.608500e+04
mean,2013.897562,6.340669,1229.010976,4.016984,1.050244,1.016562,27.719292,1.199425,1.369000,1.649275,...,2.127276,41.803552,3.203985,7.259389,6.338667,1.386027,4.038900,2014.0,0.248950,1.203414e+10
std,0.303230,3.279319,709.244089,1.907209,0.430744,0.129119,6.298364,0.428824,0.564967,1.678651,...,0.955405,72.710445,1.498200,5.943500,1.574209,3.145691,1.984006,0.0,0.432414,9.308331e+09
min,2013.000000,1.000000,0.000000,1.000000,1.000000,1.000000,12.000000,1.000000,1.000000,1.000000,...,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,2014.0,0.000000,0.000000e+00
25%,2014.000000,4.000000,729.000000,2.000000,1.000000,1.000000,23.000000,1.000000,1.000000,1.000000,...,2.000000,0.000000,2.000000,2.000000,7.000000,0.000000,2.000000,2014.0,0.000000,6.521000e+03
50%,2014.000000,6.000000,1236.000000,4.000000,1.000000,1.000000,27.000000,1.000000,1.000000,1.000000,...,2.000000,3.000000,3.000000,5.000000,7.000000,0.000000,4.000000,2014.0,0.000000,8.589941e+09
75%,2014.000000,9.000000,1749.000000,6.000000,1.000000,1.000000,32.000000,1.000000,2.000000,2.000000,...,2.000000,54.000000,5.000000,12.000000,7.000000,0.000000,6.000000,2014.0,0.000000,1.717987e+10
max,2014.000000,12.000000,9999.000000,7.000000,9.000000,3.000000,50.000000,9.000000,4.000000,29.000000,...,9.000000,364.000000,5.000000,22.000000,7.000000,9.000000,7.000000,2014.0,1.000000,2.576981e+10


In [19]:
pndf['Infant_Living'] = pndf['Infant_Living'].fillna("U")
pndf['Infant_Living'] = pndf['Infant_Living'].map({"Y": 1, "N": 0, "U": -1})
#pndf['Infant_Living'] = 0
pndf.head(20)

,Birth_Year,Birth_Month,Time_of_Birth,Birth_DOW,Birth_Place,Facility_Recode,Mothers_Age_Recode,Mothers_Nativity,Residence_Status,Mothers_Race,...,Infant_age_recode_5,Infant_age_recode_22,Manner_of_Death,Place_of_injury_for_causes_W00_Y34_except_Y06,ICD10,Day_of_Week_of_Death,Data_Year,Month_of_Death,id,Infant_Living
0,2013.0,7.0,1853.0,3.0,1.0,1.0,24.0,1,1.0,NaN,...,5.0,16.0,5.0,NaN,R99,7.0,2014.0,0.0,0,0
1,2013.0,9.0,841.0,3.0,1.0,1.0,27.0,1,1.0,NaN,...,5.0,16.0,5.0,NaN,R95,7.0,2014.0,0.0,1,0
2,2013.0,10.0,829.0,3.0,1.0,1.0,25.0,1,2.0,NaN,...,5.0,13.0,5.0,NaN,R99,4.0,2014.0,0.0,2,0
3,2013.0,10.0,1913.0,5.0,1.0,1.0,24.0,1,1.0,NaN,...,5.0,14.0,5.0,NaN,R99,4.0,2014.0,0.0,3,0
4,2013.0,10.0,2055.0,4.0,1.0,1.0,32.0,1,1.0,NaN,...,5.0,19.0,5.0,NaN,R99,5.0,2014.0,0.0,4,0
5,2013.0,11.0,2156.0,5.0,1.0,1.0,29.0,1,1.0,NaN,...,5.0,14.0,7.0,NaN,G129,1.0,2014.0,0.0,5,0
6,2013.0,11.0,11.0,2.0,1.0,1.0,19.0,1,1.0,NaN,...,5.0,14.0,7.0,NaN,A419,3.0,2014.0,0.0,6,0
7,2013.0,12.0,815.0,2.0,1.0,1.0,28.0,1,1.0,NaN,...,5.0,18.0,1.0,0.0,W75,6.0,2014.0,0.0,7,0
8,2013.0,12.0,1114.0,4.0,1.0,1.0,22.0,1,1.0,NaN,...,5.0,12.0,5.0,NaN,R95,4.0,2014.0,0.0,8,0
9,2013.0,12.0,2330.0,4.0,1.0,1.0,22.0,1,1.0,NaN,...,4.0,10.0,7.0,NaN,P77,3.0,2014.0,0.0,9,0


In [20]:

"""pndf['Infant_Gender'] = pndf['Infant_Gender'].fillna("F")
pndf['Infant_Gender'] = pndf['Infant_Gender'].map({"F": 1,"M": 2})
pndf.head(20)
"""

'pndf[\'Infant_Gender\'] = pndf[\'Infant_Gender\'].fillna("F")\npndf[\'Infant_Gender\'] = pndf[\'Infant_Gender\'].map({"F": 1,"M": 2})\npndf.head(20)\n'

In [ ]:
#find the largest cause of fetal death in 2014
# P072 is 'Extreme immaturity of newborn, unspecified weeks of gestation'
icds = pndf.groupby('ICD10')
for name, groupdf in icds:
    if groupdf['ICD10'].count() == icds.size().max():
        print(name," -- ",groupdf['ICD10'].count())

P072  --  3162


In [ ]:


#pndf_numeric = pndf[["Birth_Year", "Birth_Month", "Mothers_Age_Recode",
#                    "Delivery_Weight_lbs", "Gestational_Age_weeks", "Num_Prev_Cesareans",
#                    "Plurality", "id"]]

# calculate the correlation matrix

#sns.heatmap(pndf_numeric.corr())
sns.heatmap(pndf.corr(),annot=True,cmap='RdYlGn',linewidths=1.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(500,400)
plt.show()
fig.savefig('Correlation.png') 

In [ ]:
grpicds = icds.size()
grpicds.head()

In [ ]:
#plot of all root cause of fetal death(x-axis) and #deaths (y-axis)
import matplotlib.pyplot as plt


fig = plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')

plt.title("2014 Fetal deaths by cause ")
subplt = fig.add_subplot(1,1,1)
grpicds.plot(rot=45,grid=True ) #ax=subplt, logy=True

subplt.set_ylabel("Number of Fetal Deaths")
subplt.set_xlabel("ICD 10 codes - Root Cause of Fetal Death ")

plt.show()
fig.savefig('LeadingFetalCOD2014.png') 